In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cross_validation import train_test_split
from sklearn.datasets import make_classification
from skfeature.function.similarity_based import fisher_score

/Users/gaofan/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
# increasing n_samples
samples = [10**i for i in range(1,4)]

In [14]:
from sklearn.linear_model import LogisticRegressionCV
def comparison(sample_sizes, n_features, n_informative, n_redundant):
    selected_vars = {}
    accuracy = {}
    t_accuracy = {}
    true_vars = [i for i in range(n_informative)]
    clf = LogisticRegressionCV()
    
    for size in sample_sizes:
        print(size)
        X, y = make_classification(n_samples=size, n_features=n_features, n_informative=n_informative,
                                 n_redundant=n_redundant, n_classes=2, shuffle=False, 
                                 random_state=111, n_clusters_per_class=1)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        idx = fisher_score.fisher_score(X_train, y_train)
        pos = [list(idx).index(i) for i in range(n_informative)]
        print(pos)
        selected_vars[size] = pos
        
        clf.fit(np.array(X_train)[:,true_vars], y_train)
        t_acc = clf.score(np.array(X_test)[:,true_vars], y_test)
        print(t_acc)
        t_accuracy[size] = t_acc
        
        selected_features_fisher_train = np.array(X_train)[:, pos]
        selected_features_fisher_test = np.array(X_test)[:, pos]

        
        clf.fit(selected_features_fisher_train, y_train)
        acc = clf.score(selected_features_fisher_test, y_test)
        print(acc)
        accuracy[size] = acc
    
    return true_vars,selected_vars, accuracy, t_accuracy

In [15]:
true_vars, selected_vars, accuracies, t_accuracies = comparison(samples, 30, 3, 5)

10
[12, 1, 5]
1.0
0.5
100
[13, 10, 28]
1.0
0.6
1000
[28, 17, 7]
0.99
0.58


In [12]:
accuracies

{10: 0.0, 100: 0.9, 1000: 0.605}

In [17]:
X, y = make_classification(n_samples=1000, n_features=30, n_informative=3, n_redundant = 2, 
                           n_repeated=0, n_classes=2, shuffle=False, random_state=777, n_clusters_per_class=1)

In [134]:
#from sklearn.preprocessing import StandardScaler

In [135]:
#scaler = StandardScaler()
#X_scaled = scaler.fit_transform(X)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [137]:
# FILTER

In [19]:
idx = fisher_score.fisher_score(X_train, y_train)

In [25]:
list(idx).index(0)

15

In [23]:
print(idx)
print(len(idx))
print(idx[0:3])
pos = [list(idx).index(i) for i in range(3)]
print(pos)

[28 25 26  2  1  5 19  3 12 11 17 22 20  4 15  0 10 27  9  7 21 23 14 29
 18 16  6 24  8 13]
30
[28 25 26]
[15, 4, 3]


In [26]:
n_informative = 3
selected = np.array
pos = [list(idx).index(i) for i in range(n_informative)]
pos   

[15, 4, 3]

In [27]:
selected_features_fisher_train = np.array(X_train)[:, pos]

In [28]:
selected_features_fisher_test = np.array(X_test)[:, pos]

In [12]:
from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV()
clf.fit(X_train[:,[0,1,2]], y_train)
clf.score(X_test[:,[0,1,2]], y_test)

0.935

In [29]:
clf = LogisticRegressionCV()
clf.fit(selected_features_fisher_train, y_train)
clf.score(selected_features_fisher_test, y_test)

0.67

In [147]:
# WRAPPER

In [148]:
from sklearn.feature_selection import RFE

In [149]:
clf = LogisticRegressionCV()

In [150]:
selector = RFE(clf, 3, step=1)

In [151]:
selector = selector.fit(X_train, y_train)

In [152]:
selector.support_

array([False,  True, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False])

In [153]:
selector.score(X_test, y_test)

0.945

In [78]:
# EMBEDDED

In [154]:
from sklearn.linear_model import SGDClassifier

In [160]:
el = SGDClassifier(loss='log', penalty='elasticnet', max_iter=10000, random_state = 777, l1_ratio=0.5)

In [161]:
el.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.5, learning_rate='optimal',
       loss='log', max_iter=10000, n_iter=None, n_jobs=1,
       penalty='elasticnet', power_t=0.5, random_state=777, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [162]:
el.score(X_test, y_test)

0.94

In [163]:
el.coef_

array([[-0.31903036, -1.09126218,  0.        , -0.72954682,  0.        ,
         0.17365676,  0.        ,  0.        ,  0.        , -0.00565488,
        -0.02291722,  0.        ,  0.        ,  0.        ,  0.        ,
        -0.02813631, -0.28941577,  0.        , -0.91668643,  0.        ,
         0.36653165,  0.        , -0.58067042,  0.        ,  0.        ,
        -0.0026701 ,  0.        ,  0.1340045 ,  0.05825889, -0.10501502]])

In [164]:
np.where(el.coef_ == 0)[1]

array([ 2,  4,  6,  7,  8, 11, 12, 13, 14, 17, 19, 21, 23, 24, 26])